In [1]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras import utils
from wandb.keras import WandbCallback
import wandb

In [2]:
wandb.init(project="resume-read-group-runs", name="resume_runs", resume=True)
config = wandb.config

Streaming file created twice in same run: /home/jupyter/gcp_wandb/wandb/run-20191108_070035-hxgvzhrc/wandb-history.jsonl
Streaming file created twice in same run: /home/jupyter/gcp_wandb/wandb/run-20191108_070035-hxgvzhrc/wandb-events.jsonl


In [3]:
wandb.run.resumed

True

In [6]:
config.dropout = 0.2
config.hidden_layer_size = 128
config.layer_1_size  = 16
config.layer_2_size = 32
config.learn_rate = 0.01
config.decay = 1e-6
config.momentum = 0.9

In [4]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
labels=["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

img_width=28
img_height=28

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

#reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

# one hot encode outputs
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [7]:
# Specify the optimizer
sgd = SGD(lr=config.learn_rate, decay=config.decay, momentum=config.momentum,
                            nesterov=True)

In [7]:
# Build and compile model
def get_trainable_model():
    model = Sequential()
    model.add(Conv2D(config.layer_1_size, (5, 5), activation='relu',
                                input_shape=(img_width, img_height,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(config.layer_2_size, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(config.dropout))
    model.add(Flatten())
    model.add(Dense(config.hidden_layer_size, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    return model

In [8]:
model = get_trainable_model()
model.fit(X_train, y_train,  validation_data=(X_test, y_test), 
          epochs=25, initial_epoch=wandb.run.step,
          callbacks=[WandbCallback(data_type="image", labels=labels, save_model=True)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 13s 209us/sample - loss: 0.5707 - accuracy: 0.7897 - val_loss: 0.4157 - val_accuracy: 0.8463
Epoch 2/25
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3880 - accuracy: 0.8587 - val_loss: 0.3584 - val_accuracy: 0.8688
Epoch 3/25
60000/60000 [==============================] - 11s 181us/sample - loss: 0.3479 - accuracy: 0.8729 - val_loss: 0.3285 - val_accuracy: 0.8799
Epoch 4/25
60000/60000 [==============================] - 10s 172us/sample - loss: 0.3201 - accuracy: 0.8838 - val_loss: 0.3125 - val_accuracy: 0.8869
Epoch 5/25
60000/60000 [==============================] - 10s 172us/sample - loss: 0.3023 - accuracy: 0.8880 - val_loss: 0.3204 - val_accuracy: 0.8813
Epoch 6/25
34944/60000 [================>.............] - ETA: 4s - loss: 0.2895 - accuracy: 0.8941

KeyboardInterrupt: 

In [8]:
model = load_model(wandb.restore("model-best.h5").name)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 16)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 32)          12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 4, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               6

In [9]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [10]:
# The WandbCallback logs metrics and some example predictions on the validations data
model.fit(X_train, y_train,  validation_data=(X_test, y_test), 
          epochs=20, initial_epoch=wandb.run.step,
          callbacks=[WandbCallback(data_type="image", labels=labels, save_model=True)])

Train on 60000 samples, validate on 10000 samples
Epoch 7/20
60000/60000 [==============================] - 13s 211us/sample - loss: 0.3007 - accuracy: 0.8874 - val_loss: 0.3023 - val_accuracy: 0.8911
Epoch 8/20
60000/60000 [==============================] - 10s 173us/sample - loss: 0.2886 - accuracy: 0.8930 - val_loss: 0.2863 - val_accuracy: 0.8971
Epoch 9/20
60000/60000 [==============================] - 10s 172us/sample - loss: 0.2792 - accuracy: 0.8970 - val_loss: 0.2890 - val_accuracy: 0.8930
Epoch 10/20
60000/60000 [==============================] - 10s 173us/sample - loss: 0.2664 - accuracy: 0.9010 - val_loss: 0.2809 - val_accuracy: 0.9025
Epoch 11/20
60000/60000 [==============================] - 10s 173us/sample - loss: 0.2563 - accuracy: 0.9033 - val_loss: 0.2745 - val_accuracy: 0.9024
Epoch 12/20
60000/60000 [==============================] - 10s 173us/sample - loss: 0.2523 - accuracy: 0.9054 - val_loss: 0.2702 - val_accuracy: 0.9007
Epoch 13/20
60000/60000 [================